In [1]:
from pathlib import Path
import utils.sonar_image_analysis as iau
import importlib
importlib.reload(iau)

# Centralized sonar defaults (inserted by sweep)
from utils.sonar_config import SONAR_VIS_DEFAULTS
sonar_config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = sonar_config['range_min_m']
RANGE_MAX_M = sonar_config['range_max_m']
DISPLAY_RANGE_MAX_M = sonar_config['display_range_max_m']

# Simple Image Analysis with CV2

This notebook demonstrates:
1. **Pick a frame** from NPZ files and save it locally
2. **Use standard cv2 functions** directly for image processing
3. **Experiment** with different OpenCV operations

In [2]:
# Prefer selecting by bag ID (part of the NPZ filename) instead of a numeric index.
# Set TARGET_BAG to a substring that appears in the NPZ filename 
# '2024-08-22_14-47-39' #1
# '2024-08-22_14-29-05' #2
# '2024-08-20_14-22-12' #3
# '2024-08-20_14-31-29' #4
# '2024-08-20_18-47-40' #5
# '2024-08-20_13-55-34' #6
# '2024-08-20_13-57-42' #7
# '2024-08-20_17-02-00' #8
TARGET_BAG = '2024-08-20_17-02-00'  # change this to your desired bag ID
from utils.sonar_config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# NOTE: To export data for this specific bag, use:
# python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem 2024-08-20_17-02-00 --all

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data


## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [ ]:
# Create the unified processor
processor = iau.SonarDataProcessor()

# Use the enhanced video creation with CORE processor
video_path = iau.create_enhanced_contour_detection_video_with_processor(
    npz_file_index=NPZ_FILE_INDEX,          # Which NPZ file to use
    frame_start=1,           # Starting frame
    frame_count=1500,         # Number of frames to process
    frame_step=1,            # Step between frames
    output_path=Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / 'core_aoi_tracking.mp4',
    processor=processor      # Use our CORE processor
)

=== ENHANCED VIDEO CREATION () ===
Creating video with  processor...
Frames: 1500, step: 1
Processing 944 frames...
Processing 944 frames...
Processed 10/944 frames
Processed 10/944 frames
Processed 20/944 frames
Processed 20/944 frames
Processed 30/944 frames
Processed 30/944 frames
Processed 40/944 frames
Processed 40/944 frames
Processed 50/944 frames
Processed 50/944 frames
Processed 60/944 frames
Processed 60/944 frames
Processed 70/944 frames
Processed 70/944 frames
Processed 80/944 frames
Processed 80/944 frames
Processed 90/944 frames
Processed 90/944 frames
Processed 100/944 frames
Processed 100/944 frames
Processed 110/944 frames
Processed 110/944 frames
Processed 120/944 frames
Processed 120/944 frames
Processed 130/944 frames
Processed 130/944 frames
Processed 140/944 frames
Processed 140/944 frames
Processed 150/944 frames
Processed 150/944 frames
Processed 160/944 frames
Processed 160/944 frames
Processed 170/944 frames
Processed 170/944 frames
Processed 180/944 frames
Pr

In [ ]:
engine = iau.DistanceAnalysisEngine()
net_analysis_results = engine.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,        # Start from frame 1
    frame_count=1500,      # Analyze all frames from the video
    frame_step=1,          # Every frame
    save_outputs=True   # Save outputs to CSV and NPZ
)

=== DISTANCE ANALYSIS FROM NPZ ===
Analyzing: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_17-02-00_data_cones.npz
Processing 944 frames from 1 (step=1)
Processing 944 frames from 1 (step=1)

=== ANALYSIS COMPLETE ===
Total frames processed: 944
Successful detections: 944 (100.0%)
Distance statistics (meters):
  - Mean: 1.300
  - Std:  0.798
  - Min:  0.560
  - Max:  4.616
  - Range: 4.056
Analysis results saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_17-02-00_data_cones_analysis_results.csv

=== ANALYSIS COMPLETE ===
Total frames processed: 944
Successful detections: 944 (100.0%)
Distance statistics (meters):
  - Mean: 1.300
  - Std:  0.798
  - Min:  0.560
  - Max:  4.616
  - Range: 4.056
Analysis results saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_17-02-00_data_cones_analysis_results.csv


In [ ]:
net_analysis_results.columns

Index(['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees',
       'distance_meters', 'detection_success', 'tracking_status', 'area',
       'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation',
       'straightness', 'rect', 'centroid_x', 'centroid_y'],
      dtype='object')

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [ ]:
# Auto-detect pixel->meter mapping from the selected NPZ using utils function
mapping_info = iau.get_pixel_to_meter_mapping(selected)

# Extract commonly used variables for backwards compatibility
pixels_to_meters_avg = mapping_info['pixels_to_meters_avg']
image_shape = mapping_info['image_shape'] 
sonar_coverage_meters = mapping_info['sonar_coverage_meters']

print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")
print(f"Mapping source: {mapping_info['source']}")

Detected NPZ extent: x=[-8.660,8.660] m, y=[0.000,10.000] m
Image shape from NPZ: H=700, W=900
meters/pixel: x=0.019245, y=0.014286, avg=0.016765
Using pixels_to_meters_avg = 0.016765 m/px
Mapping source: npz_metadata


In [ ]:
iau.VisualizationEngine.plot_distance_analysis(net_analysis_results, "Real-World Distance Analysis")

In [ ]:
# COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.net_distance_analysis as sda

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.sonar_config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\n RAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"    {key}: {len(data)} records")
    else:
        print(f"    {key}: None")

print(f"\n DISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"    {key}: {data_len} records - {info['description']}")

 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_17-02-00
📡 1. Loading Navigation Data...
    Loaded 497 navigation records
    NetPitch data available: 497 valid records
📡 2. Loading Guidance Data...
    Loaded 509 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
📡 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 30 USBL records
📡 5. Loading DVL Position...
    Loaded 265 DVL position records
 6. Loading Navigation Position...
    Loaded 245 navigation position records
📡 7. Loading INS Z Position...
    INS file not found

 LOADING SUMMARY:
    Target bag: 2024-08-20_17-02-00
    Raw data loaded: 2/2
    Distance measurements: 4

 RAW DATA LOADED:
    navigation: 497 records
    guidance: 509 records

 DISTANCE MEASUREMENTS LOADED:
    USBL_3D: 30 records - 3D acoustic position
    USBL_Depth: 30 records - USBL depth measurement
    DVL_Position: 265 records - 3D DVL position
    Nav_Position: 245 records

In [ ]:
fig, comparison_stats = iau.ComparisonEngine.compare_sonar_vs_dvl(net_analysis_results, raw_data, sonar_coverage_m=sonar_coverage_meters, sonar_image_size=image_shape[0], use_plotly=True)
fig.show()


SONAR vs DVL COMPARISON STATISTICS:
Sonar mean distance: 1.300 m
DVL mean distance:   1.121 m
Scale ratio (Sonar/DVL): 1.160x
Sonar mean pitch:    -9.33
DVL mean pitch:      -1.81
Pitch difference:    -7.52
Sonar duration: 57.4s (944 frames)
DVL duration:   57.4s (497 records)


In [ ]:
# Generate Three-SYSTEM Video 
# ===========================
from pathlib import Path
import importlib

# CONFIGURATION
FFT_CSV_PATH = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose")
TARGET_FFT_FILE = FFT_CSV_PATH / f"{TARGET_BAG}_relative_pose_fft.csv"

print("GENERATING THREE-SYSTEM VIDEO WITH UNIFORM STYLING")
print("=" * 60)
print(f"Target Bag: {TARGET_BAG}")

# Reload the module to pick up the new functions
import utils.sonar_and_foto_generation as sg
importlib.reload(sg)

# Import the utility function
from utils.sonar_and_foto_generation import generate_three_system_video

try:
    # Generate video using the streamlined utility function
    video_path = generate_three_system_video(
        target_bag=TARGET_BAG,
        exports_folder=EXPORTS_FOLDER,
        net_analysis_results=net_analysis_results,
        raw_data=raw_data,
        fft_csv_path=TARGET_FFT_FILE if TARGET_FFT_FILE.exists() else None,
        start_idx=1,
        end_idx=1500
    )
    
    print(f"\nVIDEO SUMMARY:")
    print(f"   File: {video_path.name}")
    print(f"   Location: {video_path.parent}")
    print(f"   Frames: 1-1500")
    print(f"   Systems: DVL (Yellow/Orange) + FFT (Cyan) + Sonar (Magenta)")
    print(f"   Styling: Uniform line thickness and visual appearance")
    
except Exception as e:
    print(f"ideo generation failed: {e}")
    print(f"Error details: {type(e).__name__}: {str(e)}")
    import traceback
    traceback.print_exc()

GENERATING THREE-SYSTEM VIDEO WITH UNIFORM STYLING
Target Bag: 2024-08-20_17-02-00
GENERATING THREE-SYSTEM VIDEO
🔄 PREPARING THREE-SYSTEM SYNCHRONIZED DATA
✓ Using notebook 08's FFT loading methods...
✓ Loaded 410 relative FFT pose records
✓ Loaded FFT data via notebook 08 method: 410 records
✓ Using notebook 08's three-system synchronization...
Step 1: Calculating net relative positions
FFT distances appear to be in cm (max: 183.3), converting to meters
Final FFT distance range: -1.833 to 1.596 m
FFT pitch range: -82.7 to 88.1
FFT X range: -1.821 to 1.573 m
FFT Y range: -0.596 to 0.441 m
Step 2: Synchronizing all three systems
Synchronized 1851 time points
Available columns in sync_df: ['sync_timestamp', 'fft_distance_m', 'fft_pitch_deg', 'fft_x_m', 'fft_y_m', 'fft_heading', 'sonar_distance_m', 'sonar_pitch_deg', 'sonar_x_m', 'sonar_y_m', 'sonar_detection_success', 'nav_distance_m', 'nav_pitch_deg', 'nav_x_m', 'nav_y_m']
Available distance columns: ['fft_distance_m', 'sonar_distance_m